In [292]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_account.ipynb 

In [293]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a blocks summary. It only stores the `root` node of the txs merkle tree, which makes it small and constant in size. The actual mining happens on the block header.

In [294]:
class Header(Hashable):
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val): if != 'nonce': super().__setattr__('hash', sha(self.__dict__))
    def __str__(self):  return stringfy(self)

In [295]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          📉 0x63407ecf7b...623
hash:          🕊 0xe481e5a400...7ef
prev_hash:     🕌 0xe6e6ea9be7...471
number:        2
n_txs:         10
mined:         False
time:          Tue Mar 30 22:09:13 2021


If anything in the block header changes its `hash` changes automatically as well.

In [296]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

### Info

Includes all extra informations that are irrelevant for mining and therefore are not in the block header.

Calculate the total txs `volume` and `fees` included in a block.

In [315]:
def stats(txs):
    volume = sum([tx.value for tx in txs])
    fees   = sum([tx.fee   for tx in txs])
    return volume, fees

`Info` calculates some interesting facts about the `txs` that will be included in the block.

In [319]:
class Info:
    def __init__(self, txs):
        self.volume,self.fees = stats(txs)
        self.sz = get_size(txs)
    def __str__(self): return stringfy(self)

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [326]:
class Block(Hashable): 
    def __init__(self, bh, txs):
        self.info = Info(txs)
        self.bh   = bh
        self.hash = bh.hash
        self.mt   = MerkleTree(txs)
        self.txs  = self.val_txs(txs)
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),   'tx signature invalid'
        assert self.are_unique(txs),         'txs include duplicates'
        assert self.mt.root == self.bh.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __getitem__(self, key):return self.txs[key] 
    def __str__(self):         return (str(self.bh)
                                       +'\n'+str(self.info)
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(ph(tx) for tx in self.txs.keys()))

Create random list of signed txs.

In [327]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [331]:
txs = r_stxs(8)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          📖 0x70a470d949...60e
hash:          🔆 0xa00a321aee...41e
prev_hash:     📏 0x692cc64135...890
number:        2
n_txs:         8
mined:         False
time:          Tue Mar 30 22:21:11 2021
volume:        45.0 ether
fees:          0.34 ether
sz:            8658

txs:
🔵 0xcfb74df7fc...77c
🕈 0xe2d472813a...be8
💺 0x54cec9963c...1b7
🔥 0xbf67576bec...5a6
📂 0x5c616417c4...ed9
📤 0x7e3122c31f...2f9
🕗 0xf1d508e13a...d4c
💧 0x41d1af32dc...54d


Access specific tx from block with its hash.

In [332]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0xcfb74df7fc2776d75c253365ccda50aac32cf38b1c10ea3ba8e228791f79e77c'

In [333]:
tx = b[tx_hash]; print(tx)

fr:            💺 0x542CFAaaE5...F16
hash:          🔵 0xcfb74df7fc...77c
to:            📢 0x7c30d8A4a0...c63
value:         5.0 ether
fee:           0.01 ether
nonce:         6
time:          Tue Mar 30 22:21:11 2021
signed:        True
sig:           🕅 0xdf143c5ce9...4e8


Blocks can only contain unique txs. 

In [334]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [335]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected